In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor as VIF
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from ISLP import confusion_table
from sklearn.metrics import RocCurveDisplay
from dataset import vif_featuretrimmer

Load dataset and display basic information about it

In [ ]:
df = pd.read_csv('./datasets/full_cleaned_dataset.csv')
df.info()

print("Example of a row in the dataset:\n")
print(df.sample(1).to_markdown(index=False))
print("\n")
print(f"The dataset consists of {df['symbol'].nunique()} companies")
print(f"spannign from {df['year'].min()} to {df['year'].max()}\n")
print("Each row contains the cash flow statements, the balance sheets, and the income statements of a company for a given year.\n")
print("All of the companies are or were listed on the Swiss stock exchange. Companies that have been delisted, or have announced their wish to delist are marked as distressed.\n")
print(f"Out of the {df['distressed'].count()} observations, {df['distressed'].sum()} are distressed.")
print(f"This means the dataset is imbalanced, with only {df['distressed'].sum()/df['distressed'].count()*100:.2f}% of the companies being distressed.\n")

print("The dataset contains the following missing values:\n")
print(df.isna().sum().to_markdown())

We see that the dataset has a large number of features. Most of them numerical. The target variable (distressed) is categorical. However the dataset is highly imbalanced. We must find ways to deal with the imbalance.

Calcualte the Altman-Z score for each observation (where possible) and show the top and bottom 5 companies based on the Altman-Z score. Since we don't have the market cap for all companies, we cannot calculate the Altman Z-score for all companies.

In [ ]:
df_zscore = df.copy()
df_zscore["z_score"] = 1.2*(df["totalCurrentAssets"]/df["totalAssets"]) + 1.4*df["retainedEarnings"]/df["totalAssets"] + 3.3 * (df["ebitda"]-df["depreciationAndAmortization_income_statement"])/df["totalAssets"] + 0.6*df["marketcap"] / df["totalLiabilities"] + 0.999*df["revenue"]/df["totalAssets"]
df_zscore["z_score"] = df_zscore["z_score"].round(2)
df_zscore.dropna(subset=['z_score'], inplace=True)

df_zscore_print = df_zscore[['symbol', 'year', 'z_score', 'distressed']]
print(df_zscore_print.sort_values(by=['z_score'], ascending=False).head(10))
print(df_zscore_print.sort_values(by=['z_score'], ascending=True).head(10))

Let's do some general exploration of this new dataset.

In [ ]:
print("Example of a row in the dataset:\n")
print(df_zscore.sample(1).to_markdown(index=False))
print("\n")
print(f"The dataset consists of {df_zscore['symbol'].nunique()} companies")
print(f"and {df_zscore['symbol'].count()} observations")
print(f"spannign from {df_zscore['year'].min()} to {df_zscore['year'].max()}\n")
print(f"Out of the {df_zscore['distressed'].count()} observations, {df_zscore['distressed'].sum()} are distressed.")
print(f"This means the dataset is imbalanced, with only {df_zscore['distressed'].sum()/df_zscore['distressed'].count()*100:.2f}% of the companies being distressed.\n")
print(f"The Altman Z-score has a mean of {df_zscore['z_score'].mean():.2f} and a standard deviation of {df_zscore['z_score'].std():.2f}.")

print("The dataset contains the following missing values:\n")
print(df_zscore.isna().sum().to_markdown())

The Altman-Z score is usually interpreted as follows:
- Z > 3.0: Safe zone
- 1.8 < Z < 3.0: Grey zone
- Z < 1.8: Distress zone

Looking at the top and bottom 10, we get very extreme values (above 25 and below -6) which are likely due to faulty, inaccurate or missing data. These entries should be considered as outliers and removed for further analysis.
Interestingly, in the bottom 10, we don't have companies that we considered to be distressed. In a next step let's take a look at the score of the distressed companies.

In [ ]:
print(df_zscore_print.where(df_zscore_print['distressed'] == 1).sort_values(by=['z_score'], ascending=False).dropna())

We can see the the Altman-Z score does seem to be a decent indicator in the case of our distressed companies.
CSGN (Credit Suisse) does have a very low score.
STLN (Swiss Steel Holding) had a low score in 2020, but it recovered in later years. Checking the news, it shows that the company went through rebranding, after some stressful years. In 2023 it dropped again, fitting with their wish to remove themselves from the stock market.

We still have an highly imbalanced dataset. We could try to use the Z-score to define a new threshold instead of distressed. We can call it "unhealthy" and set the threshold to 3.0 for the Z-Score, then check the balance of the dateset again.

In [ ]:
df_zscore['unhealthy'] = df_zscore['z_score'].apply(lambda x: 1 if x < 3.00 else 0)

In [ ]:
print(f"Out of the {df_zscore['unhealthy'].count()} observations, {df_zscore['unhealthy'].sum()} are deemed to be in a financially unhealthy stage.")
print(f"Balance of the dataset is as follows, with {df_zscore['unhealthy'].sum()/df_zscore['unhealthy'].count()*100:.2f}% of the companies being unhealthy.\n")


Setting the thresholf to 3.0 gave us a well balanced dataset, but we have to keep in mind that we're now looking determining "unhealthy" companies, not companies that might default.

Now let's look at the distribution just numerically and visually in a histogram.

In [ ]:
print(df_zscore['z_score'].describe())
print(df_zscore['unhealthy'].value_counts().to_markdown(index=True, headers=['Unhealthy', 'Count'], tablefmt='pipe'))

# Histogram
bins =  int(math.sqrt(len(df_zscore.index))) # number of bins based on square root of number of data points
plt.hist(df_zscore['z_score'], bins=bins)
plt.title('Z-Score Distribution')
plt.xlabel('Z-Score')
plt.ylabel('Frequency')
plt.show()



This is just another proof that there are extreme outliers in the data.
Let's remove the outliers based the standard deviation (7.48) and see how the distribution looks like.

In [ ]:
std = 7.49
mean = 4.03

df_1std = df_zscore.where(
    (df_zscore['z_score'] >= mean - std) & (df_zscore['z_score'] <= mean + std)).dropna()
df_2std = df_zscore.where(
    (df_zscore['z_score'] >= mean - 2*std) & (df_zscore['z_score'] <= mean + 2*std)).dropna()
df_3std = df_zscore.where(
    (df_zscore['z_score'] >= mean - 3*std) & (df_zscore['z_score'] <= mean + 3*std)).dropna()

bins1 = int(math.sqrt(len(df_1std.index)))
bins2 = int(math.sqrt(len(df_2std.index)))
bins3 = int(math.sqrt(len(df_3std.index)))

fig, axs = plt.subplots(
    2, 3, sharey=False, tight_layout=False, figsize=(15, 10))
axs[0, 0].hist(df_1std['z_score'], bins=bins1, edgecolor='black')
axs[0, 0].set_title('1 Standard Deviation')
axs[1, 0].pie(df_1std['unhealthy'].value_counts(), labels=[
              'Healthy', 'Unhealthy'], autopct='%1.1f%%', startangle=90)
axs[0, 1].hist(df_2std['z_score'], bins=bins2, edgecolor='black')
axs[0, 1].set_title('2 Standard Deviations')
axs[1, 1].pie(df_2std['unhealthy'].value_counts(), labels=[
              'Healthy', 'Unhealthy'], autopct='%1.1f%%', startangle=90)
axs[0, 2].hist(df_3std['z_score'], bins=bins3, edgecolor='black')
axs[0, 2].set_title('3 Standard Deviations')
axs[1, 2].pie(df_3std['unhealthy'].value_counts(), labels=[
              'Healthy', 'Unhealthy'], autopct='%1.1f%%', startangle=90)
plt.show()

Staying within up to 3 standard deviations we're removing a lot of extreme cases, while still keeping a lot of the data all the while not loosing any balance.
But even after removing all the extreme observations the mean is still very high at 3.9 (for 3 stds). The expected mean should be closer to 1.8, which is also the cutoff point for the grey zone according to Altman. However, considering that the data we're working with aren't just random companies, but sourced from a high perfoming index, this might be a reasonable result. Going forward we should keep using this 3rd standard deviation set as our dataset.

Interesting are the companies with negative scores, let's take a look at them.

A correlation matrix is a good way to visualize the relationships between the variables. This might give us insight into the behind the negative Z-scores and explain how these outliers are formed.

In [ ]:
negative_scores = df_3std.where(df_3std['z_score']<0).dropna()
negative_z_score_analysis = negative_scores[['symbol', 'year', 'z_score', 'totalCurrentAssets', 'totalAssets', 'retainedEarnings', 'ebitda', 'depreciationAndAmortization_income_statement', 'totalLiabilities', 'marketcap', 'revenue']]


Since the Z-score is based on 5 ratios and not just the raw variables, we should also create a matrix with those ratios. These ratios should have a stronger correlation to the Z-score, comparitively.

In [ ]:
negative_z_score_analysis['ebit'] = negative_z_score_analysis['ebitda'] - negative_z_score_analysis['depreciationAndAmortization_income_statement']
negative_z_score_analysis = negative_z_score_analysis.drop(columns=['ebitda', 'depreciationAndAmortization_income_statement'])
negative_z_score_analysis['A'] = negative_z_score_analysis['totalCurrentAssets'] / negative_z_score_analysis['totalAssets']
negative_z_score_analysis['B'] = negative_z_score_analysis['retainedEarnings'] / negative_z_score_analysis['totalAssets']
negative_z_score_analysis['C'] = negative_z_score_analysis['ebit'] / negative_z_score_analysis['totalAssets']
negative_z_score_analysis['D'] = negative_z_score_analysis['marketcap'] / negative_z_score_analysis['totalLiabilities']
negative_z_score_analysis['E'] = negative_z_score_analysis['revenue'] / negative_z_score_analysis['totalAssets']

negative_z_score_analysis.to_csv('./datasamples/negative_z_score_analysis.csv', index=False)

In [ ]:
corr_data_raw = negative_z_score_analysis[['z_score', 'totalCurrentAssets', 'totalAssets', 'retainedEarnings', 'ebit', 'marketcap', 'totalLiabilities', 'revenue']]
corr_data_z = negative_z_score_analysis[['z_score','A', 'B', 'C', 'D', 'E']]

matrix1 = corr_data_raw.corr(numeric_only=True)
matrix2 = corr_data_z.corr(numeric_only=True)

fig, axs = plt.subplots(ncols=2, figsize=(15, 5))
sns.heatmap(matrix1, annot=True, ax=axs[0])
sns.heatmap(matrix2, annot=True, ax=axs[1])
plt.show()

Now we have to matrices showing correlations between the Altman Z-Score and the datapoints of the companies with negative scores.
The first matrix show the correlations of the datapoints as given in the dataset.
The second matrix shows the correlations of the ratios used by the Altman Z-Score calculation.
Here we see that 'B', the retained Earnings to total Assets ratio has the strongest correlation to the z-score.

Looking at the data, we see that all retained earnings are negative in this subset.

Let's create the second matrix using the entire dataset, and compare two matrices.

In [ ]:
complete_z_score_analysis = df_3std[['symbol', 'year', 'z_score', 'totalCurrentAssets', 'totalAssets', 'retainedEarnings', 'ebitda', 'depreciationAndAmortization_income_statement', 'totalLiabilities', 'marketcap', 'revenue']]
complete_z_score_analysis['ebit'] = complete_z_score_analysis['ebitda'] - complete_z_score_analysis['depreciationAndAmortization_income_statement']
complete_z_score_analysis = complete_z_score_analysis.drop(columns=['ebitda', 'depreciationAndAmortization_income_statement'])
complete_z_score_analysis['A'] = complete_z_score_analysis['totalCurrentAssets'] / complete_z_score_analysis['totalAssets']
complete_z_score_analysis['B'] = complete_z_score_analysis['retainedEarnings'] / complete_z_score_analysis['totalAssets']
complete_z_score_analysis['C'] = complete_z_score_analysis['ebit'] / complete_z_score_analysis['totalAssets']
complete_z_score_analysis['D'] = complete_z_score_analysis['marketcap'] / complete_z_score_analysis['totalLiabilities']
complete_z_score_analysis['E'] = complete_z_score_analysis['revenue'] / complete_z_score_analysis['totalAssets']

complete_corr_data = complete_z_score_analysis[['z_score','A', 'B', 'C', 'D', 'E']]

complete_matrix = complete_corr_data.corr(numeric_only=True)
negative_matrix = corr_data_z.corr(numeric_only=True)

fig, axs = plt.subplots(ncols=2, figsize=(15, 5))
sns.heatmap(complete_matrix, annot=True, ax=axs[0])
sns.heatmap(negative_matrix, annot=True, ax=axs[1])
axs[0].set_title('All Z-Score')
axs[1].set_title('Negative Z-Score')
plt.show()

In comparison, the correlation of B is weaker in the full dataset, but it's still decently strong.
The strongest correlation is now D, Market Cap to Total Liabilities ratio. Which I believe logically makes sense, as we're dealing with companies that have been sourced from a high performing index.

Another way to check the data for multicollinearity (not correlation) is to use the variance inflaition factor (VIF).
The VIF cannot deal with empty values, so we will remove them first.

The main dataset has many empty values.
The 3-standard-deviation set has no empty, since the z-score calculation and removal of outliers presumably dealt with them. We will also craete a standardized set for comparison. Since the values in the set have a lot of variance, sttandardizing should hlep with calculations.

In [ ]:
scaler = StandardScaler()
#prepare data
#load data to do the VIF analysis
df_3std_vif = df_3std.copy()


#drop columns not required
df_3std_vif.drop(columns=['z_score', 'year'], inplace=True)

#drop missing values (should do nothing, but just in case)
df_3std_vif.dropna(inplace=True)

#get list of numeric columns, and drop non-numeric columns
numeric_columns = df_3std_vif.select_dtypes(include=[np.number]).columns.tolist()
df_3std_vif = df_3std_vif[numeric_columns]

In [ ]:
# calculate VIFs
vals1 = [VIF(df_3std_vif, i) for i in range(1, df_3std_vif.shape[1])]
vif = pd.DataFrame({"vif": vals1}, index=df_3std_vif.columns[1:])
vif = vif.sort_values("vif", ascending=False)

vif = vif.round(3)

print(vif.to_markdown())

We notice that there are many ridiculously high VIF scores.
Taking a closer look at those values we notice that many of the calculated values (totals, ratios) and the values used for their calculation are part of the group of high VIFs. Let's iterively drop the highest VIF until we reach a reasonable level of VIFs.

In [ ]:
df_3std_vif_drop = df_3std.copy()

#drop unnecessary columns
df_3std_vif_drop.drop(columns=['z_score', 'year'], inplace=True)

# note: calculation can take up to 5 minutes
df_3std_trimmed = vif_featuretrimmer(df_3std_vif_drop, 20)

After removing a large number of features, with a threshold of 10, we are left with 44 features (including 2 possible intercepts in distressed and unhealthy) to create a logistic regression model with.

Let's run a logistic regression.

In [ ]:
# create y, the target variable and X, the features
y = list(df_3std_trimmed['unhealthy'])
numeric_columns = df_3std_trimmed.select_dtypes(include=[np.number]).columns.tolist()
X = df_3std_trimmed[numeric_columns]

# dropping targets
X = X.drop(columns=['distressed', 'unhealthy'])
# dropping high correlation features that VIF didn't catch
X = X.drop(columns=['grossProfitRatio', 'ebitdaratio', 'epsdiluted'])
# dropping features as part of model selection
X = X.drop(columns=['researchAndDevelopmentExpenses','shortTermDebt','sellingAndMarketingExpenses','acquisitionsNet','taxPayables','deferredRevenueNonCurrent','otherWorkingCapital','otherAssets','commonStock','effectOfForexChangesOnCash','propertyPlantEquipmentNet','otherCurrentAssets','capitalLeaseObligations','deferredIncomeTax','weightedAverageShsOut', 'weightedAverageShsOutDil', 'commonStockRepurchased', 'othertotalStockholdersEquity'])

# run the logistic regression model
glm = sm.GLM(y, X, family=sm.families.Binomial())
results = glm.fit()

print(results.summary())

After doing model selection we now have a decent model. 

In [ ]:
#save the model
results.save('models/log_01.pkl')

In [ ]:
probs = results.predict(X)
pred_labels = np.array(['0.0']*712)
pred_labels[probs>0.5] = '1.0'

true_labels = df_3std_vif['unhealthy'].values.astype(str)

confusion_table(pred_labels, true_labels)

In [ ]:
roc_curve = RocCurveDisplay.from_predictions

roc_curve(true_labels, probs, pos_label='1.0', name='unhealthy')

The ROC curve also shows that the model is decent at predicting "unhealthy" companies. However, there is a chance that this is due to the model overfitting. We should separate the data into a training and test set, rerun the model and evaluate the results. We could randomly split the data, but since we do have a time series, we should split the data by time. We can put all data from 2024 into the test set and use the rest for training.

In [ ]:
# create test and train datasets
df_test = df_3std[df_3std['year'] == 2024].copy()
df_train = df_3std[df_3std['year'] != 2024].copy()

Let's clean up the data with the same process as before, before running the logistic regression.

In [ ]:
numeric_columns = df_train.select_dtypes(include=[np.number]).columns.tolist()
df_train = df_train[numeric_columns]

#drop unnecessary columns
df_train.drop(columns=['z_score', 'year'], inplace=True)

# note: calculation can take up to 5 minutes
df_train_viftrim = vif_featuretrimmer(df_train, 20)

In [ ]:
# create y, the target variable and X, the features
y = list(df_train_viftrim['unhealthy'])
numeric_columns = df_train_viftrim.select_dtypes(include=[np.number]).columns.tolist()
X = df_train_viftrim[numeric_columns]

# dropping targets
X = X.drop(columns=['distressed', 'unhealthy'])
# dropping high correlation features that VIF didn't catch
X = X.drop(columns=['otherNonCurrentAssets', 'grossProfitRatio', 'ebitdaratio'])
# dropping features as part of model selection
X = X.drop(columns=['deferredRevenue','shortTermInvestments','salesMaturitiesOfInvestments','intangibleAssets','deferredIncomeTax','depreciationAndAmortization_income_statement','accumulatedOtherComprehensiveIncomeLoss','deferredRevenueNonCurrent','accountsReceivables','effectOfForexChangesOnCash','taxPayables','weightedAverageShsOutDil','totalCurrentLiabilities'])
# run the logistic regression model
glm = sm.GLM(y, X, family=sm.families.Binomial())
results = glm.fit()

print(results.summary())

In [ ]:
#save the model
results.save('models/log_02.pkl')

In [ ]:
#remove columns in the test set to match the train set
columns_tokeep = [
    "cashAndCashEquivalents",
    "otherCurrentAssets",
    "deferredTaxLiabilitiesNonCurrent",
    "capitalLeaseObligations",
    "preferredStock",
    "commonStock",
    "othertotalStockholdersEquity",
    "minorityInterest",
    "stockBasedCompensation",
    "inventory_cash_flow_statement",
    "accountsPayables",
    "acquisitionsNet",
    "commonStockRepurchased",
    "otherFinancingActivites",
    "costOfRevenue",
    "researchAndDevelopmentExpenses",
    "generalAndAdministrativeExpenses",
    "sellingAndMarketingExpenses",
    "operatingExpenses",
    "operatingIncome",
    "totalOtherIncomeExpensesNet",
    "incomeTaxExpense",
    "weightedAverageShsOut"
]

df_test_model = df_test[columns_tokeep].copy()

In [ ]:
probs = results.predict(exog=df_test_model)
true_labels = df_test['unhealthy'].values.astype(str)

roc_curve(true_labels, probs, pos_label='1.0', name='unhealthy')

On the test set we the model ends up being a bit worse.